<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# DKN : Deep Knowledge-Aware Network for News Recommendation
DKN \[1\] is a deep learning model which incorporates information from knowledge graph for better news recommendation. Specifically, DKN uses TransX \[2\] method for knowledge graph representaion learning, then applies a CNN framework, named KCNN, to combine entity embedding with word embedding and generate a final embedding vector for a news article. CTR prediction is made via an attention-based neural scorer. 

## Properties of DKN:
- DKN is a content-based deep model for CTR prediction rather than traditional ID-based collaborative filtering. 
- It makes use of knowledge entities and common sense in news content via joint learning from semantic-level and knnowledge-level representations of news articles.
- DKN uses an attention module to dynamically calculate a user's aggregated historical representaition.

## Reference environment 
we tested this notebook with two environment settings:
cpu with tensorflow==1.15.2 and gpu with tensorflow==1.15.2 on July 1, 2020.

## Data format:
### DKN takes several files as input as follows:
- training / validation / test files: each line in these files represents one instance. Impressionid is used to evaluate performance within an impression session, so it is only used when evaluating, you can set it to 0 for training data. The format is : <br> 
`[label] [userid] [CandidateNews]%[impressionid] `<br> 
e.g., `1 train_U1 N1%0` <br> 
- user history file: each line in this file represents a users' click history. You need to set his_size parameter in config file, which is the max number of user's click history we use. We will automatically keep the last his_size number of user click history, if user's click history is more than his_size, and we will automatically padding 0 if user's click history less than his_size. the format is : <br> 
`[Userid] [newsid1,newsid2...]`<br>
e.g., `train_U1 N1,N2` <br> 
- document feature file:
It contains the word and entity features of news. News article is represented by (aligned) title words and title entities. To take a quick example, a news title may be : Trump to deliver State of the Union address next week , then the title words value may be CandidateNews:34,45,334,23,12,987,3456,111,456,432 and the title entitie value may be: entity:45,0,0,0,0,0,0,0,0,0. Only the first value of entity vector is non-zero due to the word Trump. The title value and entity value is hashed from 1 to n(n is the number of distinct words or entities). Each feature length should be fixed at k(doc_size papameter), if the number of words in document is more than k, you should truncate the document to k words, and if the number of words in document is less than k, you should padding 0 to the end. 
the format is like: <br> 
`[Newsid] [w1,w2,w3...wk] [e1,e2,e3...ek]`
- word embedding/entity embedding/ context embedding files: These are npy files of pretrained embeddings. After loading, each file is a [n+1,k] two-dimensional matrix, n is the number of words(or entities) of their hash dictionary, k is dimension of the embedding, note that we keep embedding 0 for zero padding. 
In this experiment, we used GloVe\[4\] vectors to initialize the word embedding. We trained entity embedding using TransE\[2\] on knowledge graph and context embedding is the average of the entity's neighbors in the knowledge graph.<br>

## Global settings and imports

In [1]:
import sys
sys.path.append("../../")
from reco_utils.recommender.deeprec.deeprec_utils import *
from reco_utils.recommender.deeprec.models.dkn import *
from reco_utils.recommender.deeprec.io.dkn_iterator import *
import papermill as pm


## Download and load data

In [2]:
data_path = '../../tests/resources/deeprec/dkn'
yaml_file = os.path.join(data_path, r'dkn.yaml')
train_file = os.path.join(data_path, r'train_mind_demo.txt')
valid_file = os.path.join(data_path, r'valid_mind_demo.txt')
test_file = os.path.join(data_path, r'test_mind_demo.txt')
news_feature_file = os.path.join(data_path, r'doc_feature.txt')
user_history_file = os.path.join(data_path, r'user_history.txt')
wordEmb_file = os.path.join(data_path, r'word_embeddings_100.npy')
entityEmb_file = os.path.join(data_path, r'TransE_entity2vec_100.npy')
contextEmb_file = os.path.join(data_path, r'TransE_context2vec_100.npy')
infer_embedding_file = os.path.join(data_path, r'infer_embedding.txt')
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.blob.core.windows.net/deeprec/', data_path, 'mind-demo.zip')
    
    

## Create hyper-parameters

In [3]:
epoch=5

In [4]:
hparams = prepare_hparams(yaml_file,
                          news_feature_file = news_feature_file,
                          user_history_file = user_history_file,
                          wordEmb_file=wordEmb_file,
                          entityEmb_file=entityEmb_file,
                          contextEmb_file=contextEmb_file,
                          epochs=epoch)
print(hparams)

[('DNN_FIELD_NUM', None), ('EARLY_STOP', 100), ('FEATURE_COUNT', None), ('FIELD_COUNT', None), ('L', None), ('MODEL_DIR', None), ('PAIR_NUM', None), ('SUMMARIES_DIR', None), ('T', None), ('activation', ['sigmoid']), ('att_fcn_layer_sizes', None), ('attention_activation', 'relu'), ('attention_dropout', 0.0), ('attention_layer_sizes', 100), ('attention_size', None), ('batch_size', 100), ('cate_embedding_dim', None), ('cate_vocab', None), ('contextEmb_file', '../../tests/resources/deeprec/dkn\\TransE_context2vec_100.npy'), ('cross_activation', 'identity'), ('cross_l1', 0.0), ('cross_l2', 0.0), ('cross_layer_sizes', None), ('cross_layers', None), ('data_format', 'dkn'), ('dim', 100), ('doc_size', 10), ('dropout', [0.0]), ('dtype', 32), ('embed_l1', 0.0), ('embed_l2', 1e-06), ('embedding_dropout', 0.3), ('enable_BN', False), ('entityEmb_file', '../../tests/resources/deeprec/dkn\\TransE_entity2vec_100.npy'), ('entity_dim', 100), ('entity_embedding_method', 'TransE'), ('entity_size', 1406), (

In [5]:
input_creator = DKNTextIterator

## Train the DKN model

In [6]:
model = DKN(hparams, input_creator)

In [7]:
print(model.run_eval(valid_file))

{'group_auc': 0.5, 'mean_mrr': 0.0759, 'ndcg@5': 0.0567, 'ndcg@10': 0.0783}


In [8]:
model.fit(train_file, valid_file)

at epoch 1
train info: logloss loss:0.6025281803948539
eval info: group_auc:0.5726, mean_mrr:0.2035, ndcg@10:0.2689, ndcg@5:0.203
at epoch 1 , train time: 537.5 eval time: 1068.0
at epoch 2
train info: logloss loss:0.5871794990130833
eval info: group_auc:0.5789, mean_mrr:0.2123, ndcg@10:0.2756, ndcg@5:0.2076
at epoch 2 , train time: 473.6 eval time: 984.5
at epoch 3
train info: logloss loss:0.5733468277113778
eval info: group_auc:0.584, mean_mrr:0.2182, ndcg@10:0.2847, ndcg@5:0.2121
at epoch 3 , train time: 518.1 eval time: 830.4
at epoch 4
train info: logloss loss:0.5610611098153251
eval info: group_auc:0.5956, mean_mrr:0.2231, ndcg@10:0.2869, ndcg@5:0.2158
at epoch 4 , train time: 456.8 eval time: 1002.5
at epoch 5
train info: logloss loss:0.5502915297235761
eval info: group_auc:0.601, mean_mrr:0.2257, ndcg@10:0.2882, ndcg@5:0.2177
at epoch 5 , train time: 398.1 eval time: 956.4


Now we can test again the performance on valid set:

In [9]:
res = model.run_eval(valid_file)
print(res)
pm.record("res", res)

{'group_auc': 0.601, 'mean_mrr': 0.2257, 'ndcg@5': 0.2177, 'ndcg@10': 0.2882}


## Document embedding inference API
After training, you can get document embedding through this document embedding inference API. The input file format is same with document feature file. The output file fomrat is: `[Newsid] [embedding]`

In [10]:
model.run_get_embedding(news_feature_file, infer_embedding_file)

## Running models with large dataset
Here are performances using the whole MIND dataset \[3\]. 

MIND dataset is a large-scale English news dataset. It was collected from anonymized behavior logs of Microsoft News website. MIND contains 1,000,000 users, 161,013 news articles and 15,777,377 impression logs. Every news article contains rich textual content including title, abstract, body, category and entities. Each impression log contains the click events, non-clicked events and historical news click behaviors of this user before this impression.

| Models | g-AUC | MRR |NDCG@5 | NDCG@10 |
| :------| :------: | :------: | :------: | :------ |
| LibFM | 0.5993 | 0.2823 | 0.3005 | 0.3574 |
| Wide&Deep | 0.6216 | 0.2931 | 0.3138 | 0.3712 |
| DKN | 0.6436 | 0.3128 | 0.3371 | 0.3908|


Note that the results of DKN is using Microsoft recommender and the results of first two models is from MIND paper \[3\].
We compare the results on the same test dataset. 

One epoch takes 6381.3s(5066.6s for training, 1314.7s for evaluating) for DKN on GPU. Hardware specification for running the large dataset: <br>
GPU: Tesla P100-PCIE-16GB <br>
CPU: 6 cores Intel(R) Xeon(R) CPU E5-2690 v4 @ 2.60GHz

## Reference
\[1\] Wang, Hongwei, et al. "DKN: Deep Knowledge-Aware Network for News Recommendation." Proceedings of the 2018 World Wide Web Conference on World Wide Web. International World Wide Web Conferences Steering Committee, 2018.<br>
\[2\] Knowledge Graph Embeddings including TransE, TransH, TransR and PTransE. https://github.com/thunlp/KB2E <br>
\[3\] Wu, Fangzhao, et al. "MIND: A Large-scale Dataset for News Recommendation" Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics. https://msnews.github.io/competition.html <br>
\[4\] GloVe: Global Vectors for Word Representation. https://nlp.stanford.edu/projects/glove/